In [83]:
import numpy as np
import pandas as pd
from datetime import datetime
import pytz
# entweder einen punkt raussuchen und dafür predictions machen
# oder deutschland in vier zonen aufteilen und für jede zone predictions machen

# kapazität ins model mitgeben

# z-normalisierung machen

# winddaten: u² und v² (addieren und wurzel ziehen?)

# cross validation für ein jahr machen

In [84]:
longitude = 7.8
latitude = 52.2

# load data
weather_data = pd.read_csv('data/Weather_Data_Germany.csv')
weather_data_2022 = pd.read_csv('data/Weather_Data_Germany_2022.csv')
supply_germany = pd.read_csv('data/Realised_Supply_Germany.csv', delimiter=';', decimal=',', thousands='.')
installed_capacity = pd.read_csv('data/Installed_Capacity_Germany.csv', delimiter=';', decimal=',', thousands='.')
installed_capacity_date = pd.read_csv('data/Installed_Capacity_Germany.csv', delimiter=';')

In [85]:

# filter data
#weather_data = weather_data[(weather_data['longitude'] == longitude) & (weather_data['latitude'] == latitude)]
#weather_data_2022 = weather_data_2022[(weather_data_2022['longitude'] == longitude) & (weather_data_2022['latitude'] == latitude)]

# divide germany into four zones
half_latitude = 51.0
half_longitude = 10.0

# north west germany: latitude > half_latitude, longitude < half_longitude
weather_data = weather_data[(weather_data['latitude'] > half_latitude) & (weather_data['longitude'] < half_longitude)]
weather_data_2022 = weather_data_2022[(weather_data_2022['latitude'] > half_latitude) & (weather_data_2022['longitude'] < half_longitude)]
# north east germany: latitude > half_latitude, longitude > half_longitude
#weather_data = weather_data[(weather_data['latitude'] > half_latitude) & (weather_data['longitude'] > half_longitude)]
#weather_data_2022 = weather_data_2022[(weather_data_2022['latitude'] > half_latitude) & (weather_data_2022['longitude'] > half_longitude)]
# south west germany: latitude < half_latitude, longitude < half_longitude
#weather_data = weather_data[(weather_data['latitude'] < half_latitude) & (weather_data['longitude'] < half_longitude)]
#weather_data_2022 = weather_data_2022[(weather_data_2022['latitude'] < half_latitude) & (weather_data_2022['longitude'] < half_longitude)]

# south east germany: latitude < half_latitude, longitude > half_longitude
#weather_data = weather_data[(weather_data['latitude'] < half_latitude) & (weather_data['longitude'] > half_longitude)]
#weather_data_2022 = weather_data_2022[(weather_data_2022['latitude'] < half_latitude) & (weather_data_2022['longitude'] > half_longitude)]



print(weather_data.columns)
# drop irrelevant columns
weather_data = weather_data.drop(columns=['longitude','latitude','forecast_origin', 'cdir', 'z',
       'blh', 'tsr', 'tp', 'fsr'])
weather_data_2022 = weather_data_2022.drop(columns=['longitude','latitude','forecast_origin', 'cdir', 'z',
       'blh', 'tsr', 'tp', 'fsr'])

# take mean for all data points where time is equal
weather_data = weather_data.groupby('time').mean().reset_index()
weather_data_2022 = weather_data_2022.groupby('time').mean().reset_index()

# drop all columns except Date from, Solar, Wind
print(supply_germany.columns)
supply_germany = supply_germany.drop(columns=['Date to', 'Biomass [MW]', 'Hydro Power [MW]', 'Other Renewable [MW]',
       'Nuclear Power [MW]', 'Lignite [MW]', 'Coal [MW]', 'Natural Gas [MW]',
       'Pumped Storage [MW]', 'Other Conventional [MW]'])


print(supply_germany.head())


installed_capacity = installed_capacity.drop(columns=['Date to', 'Biomass [MW]', 'Hydro Power [MW]', 'Other Renewable [MW]',
       'Nuclear Power [MW]', 'Lignite [MW]', 'Coal [MW]', 'Natural Gas [MW]',
       'Pumped Storage [MW]', 'Other Conventional [MW]'])
installed_capacity_date = installed_capacity_date.drop(columns=['Date to','Biomass [MW]', 'Hydro Power [MW]', 'Other Renewable [MW]',
       'Nuclear Power [MW]', 'Lignite [MW]', 'Coal [MW]', 'Natural Gas [MW]',
       'Pumped Storage [MW]', 'Other Conventional [MW]'])
print(installed_capacity.head())


Index(['longitude', 'latitude', 'forecast_origin', 'time', 'cdir', 'z', 'msl',
       'blh', 'tcc', 'u10', 'v10', 't2m', 'ssr', 'tsr', 'sund', 'tp', 'fsr',
       'u100', 'v100'],
      dtype='object')
Index(['Date from', 'Date to', 'Biomass [MW]', 'Hydro Power [MW]',
       'Wind Offshore [MW] ', 'Wind Onshore [MW]', 'Photovoltaic [MW]',
       'Other Renewable [MW]', 'Nuclear Power [MW]', 'Lignite [MW]',
       'Coal [MW]', 'Natural Gas [MW]', 'Pumped Storage [MW]',
       'Other Conventional [MW]'],
      dtype='object')
        Date from  Wind Offshore [MW]   Wind Onshore [MW]  Photovoltaic [MW]
0  01.01.19 00:00               794.25            4839.25                0.0
1  01.01.19 00:15               793.50            5030.75                0.0
2  01.01.19 00:30               791.75            5213.50                0.0
3  01.01.19 00:45               754.50            5308.25                0.0
4  01.01.19 01:00               715.50            5398.50                0.0
   Date 

In [86]:

# convert supply germany to greenwich time gmt
supply_germany['Date from'] = pd.to_datetime(supply_germany['Date from'], format='%d.%m.%y %H:%M')
# handle daylight saving time
supply_germany['Date from'] = supply_germany['Date from'].dt.tz_localize('Europe/Berlin', ambiguous='NaT').dt.tz_convert('GMT')
supply_germany['Date from'] = supply_germany['Date from'].dt.tz_localize(None)
# manually insert daylight saving time

daylight_saving_days = ["2019-10-27", "2020-10-25", "2021-10-31", "2022-10-30"]
# transform to datetime
old_format = '%Y-%m-%d'
new_format = '%d.%m.%y'
daylight_saving_days = pd.to_datetime(daylight_saving_days, format='%Y-%m-%d')
# minus 15 minutes
daylight_saving_days = daylight_saving_days - pd.Timedelta(minutes=15)
print(daylight_saving_days)

# find indices of daylight saving days
supply_germany['Date from'] = pd.to_datetime(supply_germany['Date from'])
indices = supply_germany.index[supply_germany['Date from'].isin(daylight_saving_days)].tolist()

# add time index to missing indices
for i in indices:
    for missing_index in range(0, 9):
        supply_germany.loc[i + missing_index, 'Date from'] = supply_germany['Date from'].iloc[i] + pd.Timedelta(minutes=15) * (missing_index)

supply_germany = supply_germany.set_index('Date from')

# make 15 minute timesteps to hourly timesteps
# add values of 4 15 minute timesteps to get hourly values
supply_germany = supply_germany.resample('h').sum()
# add 1 hour to the time index
supply_germany.index = supply_germany.index + pd.Timedelta(hours=1)


DatetimeIndex(['2019-10-26 23:45:00', '2020-10-24 23:45:00',
               '2021-10-30 23:45:00', '2022-10-29 23:45:00'],
              dtype='datetime64[ns]', freq=None)


In [87]:
# wind speed
weather_data['ws10'] = np.sqrt(weather_data['u10']**2 + weather_data['v10']**2)
weather_data_2022['ws10'] = np.sqrt(weather_data_2022['u10']**2 + weather_data_2022['v10']**2)
weather_data = weather_data.drop(columns=['u10', 'v10'])
weather_data_2022 = weather_data_2022.drop(columns=['u10', 'v10'])

weather_data['ws100'] = np.sqrt(weather_data['u100']**2 + weather_data['v100']**2)
weather_data_2022['ws100'] = np.sqrt(weather_data_2022['u100']**2 + weather_data_2022['v100']**2)
weather_data = weather_data.drop(columns=['u100', 'v100'])
weather_data_2022 = weather_data_2022.drop(columns=['u100', 'v100'])

# convert to datetime
weather_data['time'] = pd.to_datetime(weather_data['time'], format='%Y-%m-%d %H:%M:%S')
weather_data_2022['time'] = pd.to_datetime(weather_data_2022['time'], format='%Y-%m-%d %H:%M:%S')

weather_data = weather_data.set_index('time')
weather_data_2022 = weather_data_2022.set_index('time')



In [88]:
# delete dots from values

# take index of installed_capacity_date and values from installed_capacity
installed_capacity['Date from'] = installed_capacity_date['Date from']

print(installed_capacity.head())
# to datetime
installed_capacity['Date from'] = pd.to_datetime(installed_capacity['Date from'], format='%d.%m.%y')

# as index
installed_capacity = installed_capacity.set_index('Date from')
# resample to hourly values

#rename columns with 'max'
installed_capacity = installed_capacity.rename(columns={'Wind Offshore [MW] ': 'max Wind Offshore [MW]'})
installed_capacity = installed_capacity.rename(columns={'Wind Onshore [MW]': 'max Wind Onshore [MW]'})
installed_capacity = installed_capacity.rename(columns={'Photovoltaic [MW]': 'max Photovoltaic [MW]'})


  Date from  Wind Offshore [MW]   Wind Onshore [MW]  Photovoltaic [MW]
0  01.01.19               6393.0            52394.0            44036.0
1  01.01.20               7504.0            53553.0            47754.0
2  01.01.21               7774.0            54345.0            50410.0
3  01.01.22               7787.0            55289.0            57744.0


In [89]:
# get last value of installed capacity
installed_capacity_2022 = installed_capacity.loc['2022-01-01':'2022-12-31']
installed_capacity_2022 = pd.DataFrame(installed_capacity_2022, index=pd.date_range(start='2022-01-01', end='2023-01-01', freq='H'))
installed_capacity_2022 = installed_capacity_2022.ffill()
installed_capacity_2022 = installed_capacity_2022[:-1]
print(installed_capacity_2022)

installed_capacity = installed_capacity.resample('h').ffill()
print(installed_capacity)

# 2019-2021
installed_capacity = installed_capacity.loc['2019-01-01':'2021-12-31']



                     max Wind Offshore [MW]  max Wind Onshore [MW]  \
2022-01-01 00:00:00                  7787.0                55289.0   
2022-01-01 01:00:00                  7787.0                55289.0   
2022-01-01 02:00:00                  7787.0                55289.0   
2022-01-01 03:00:00                  7787.0                55289.0   
2022-01-01 04:00:00                  7787.0                55289.0   
...                                     ...                    ...   
2022-12-31 19:00:00                  7787.0                55289.0   
2022-12-31 20:00:00                  7787.0                55289.0   
2022-12-31 21:00:00                  7787.0                55289.0   
2022-12-31 22:00:00                  7787.0                55289.0   
2022-12-31 23:00:00                  7787.0                55289.0   

                     max Photovoltaic [MW]  
2022-01-01 00:00:00                57744.0  
2022-01-01 01:00:00                57744.0  
2022-01-01 02:00:00     

/tmp/ipykernel_23496/2720594546.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  installed_capacity_2022 = pd.DataFrame(installed_capacity_2022, index=pd.date_range(start='2022-01-01', end='2023-01-01', freq='H'))


In [90]:
print(installed_capacity.shape)
print(weather_data.shape)
print(supply_germany.shape)
print(installed_capacity_2022.shape)
print(weather_data_2022.shape)

# add column with number of day in the year to weather data
weather_data['day'] = weather_data.index.dayofyear
weather_data_2022['day'] = weather_data_2022.index.dayofyear

# add column with hour of the day to weather data
weather_data['hour'] = weather_data.index.hour
weather_data_2022['hour'] = weather_data_2022.index.hour

#make them at beginning of the dataframe
weather_data = weather_data[['day', 'hour'] + [col for col in weather_data.columns if col not in ['day', 'hour']]]
weather_data_2022 = weather_data_2022[['day', 'hour'] + [col for col in weather_data_2022.columns if col not in ['day', 'hour']]]


# merge installed capacity with weather data by time
training_features = pd.merge(weather_data, installed_capacity, left_index=True, right_index=True)
testing_features = pd.merge(weather_data_2022, installed_capacity_2022, left_index=True, right_index=True)

# z normalization
# mean and std of training data per colums
mean = training_features.mean(axis=0)
std = training_features.std(axis=0)


training_features = (training_features - mean) / std
testing_features = (testing_features - mean) / std

# split supply germany into training and testing data
training_supply = supply_germany.loc['2019-01-01':'2021-12-31']
testing_supply = supply_germany.loc['2022-01-01':'2022-12-31']

# merge supply germany with weather data by time
training_data = pd.merge(training_features, training_supply, left_index=True, right_index=True)
testing_data = pd.merge(testing_features, testing_supply, left_index=True, right_index=True)

# add column with year
training_data['year'] = training_data.index.year
testing_data['year'] = testing_data.index.year


print(training_data.head())


(26304, 3)
(26304, 7)
(35064, 3)
(8760, 3)
(8760, 7)
                          day      hour       msl       tcc       t2m  \
time                                                                    
2019-01-01 00:00:00 -1.727278 -1.661293  1.267299  1.004777 -0.508498   
2019-01-01 01:00:00 -1.727278 -1.516833  1.201893  0.911616 -0.510114   
2019-01-01 02:00:00 -1.727278 -1.372373  1.130893  0.991098 -0.526276   
2019-01-01 03:00:00 -1.727278 -1.227912  1.063015  1.187727 -0.523993   
2019-01-01 04:00:00 -1.727278 -1.083452  0.980189  1.204908 -0.516030   

                          ssr      sund      ws10     ws100  \
time                                                          
2019-01-01 00:00:00 -0.811937 -0.912707  1.116571  1.128217   
2019-01-01 01:00:00 -0.811937 -0.912707  1.311259  1.317276   
2019-01-01 02:00:00 -0.811937 -0.912707  1.499970  1.521345   
2019-01-01 03:00:00 -0.811937 -0.912707  1.662344  1.688735   
2019-01-01 04:00:00 -0.811937 -0.912707  1.676910  1.6942

In [91]:

# save data
weather_data.to_csv('data/Weather_Data_Northwest.csv', index=False)
weather_data_2022.to_csv('data/Weather_Data_Northwest_2022.csv', index=False)



# save supply germany to csv
supply_germany.to_csv('data/Realised_Supply_Germany_GMT.csv')

training_data.to_csv('data/Training_Data_Northwest.csv')
testing_data.to_csv('data/Testing_Data_Northwest.csv')
